# How to create a Saliency Map using saliency_map.py

Saliency_map.py implements a Saliency Map handler to create saliency maps for RNN Encoder-Decoder forecasting models. It uses the method of perturbation to find out which features are the most important for the model. It also takes the time dimension into account, so highlight the most important timesteps.

The End result is a colour coded two dimensional Time x Feature heatmap. Red signifies important features and time steps, while blue signifies, that it's not important for the forecast, wheter or not the parameter is noisy.

## Choosing a target

First of all, the data on which the evaluation should take place has to be chosen. Put your data in the 'proloaf/data' folder. In this example, data from the "open opwer system data" website is used (https://data.open-power-system-data.org/time_series/).

## Train your model

if the model has already been trained and saved, you can skip this step

Create a folder for the model in 'targets' and set the configurations in the "config.json". The example config file looks like this: 

In [ ]:
{
    "data_path": "./data/opsd.csv",
    "output_path": "./oracles/",
    "exploration_path": "./targets/opsd/tuning.json",
    "evaluation_path": "./oracles/eval_opsd_recurrent/",
    "log_path": "./logs/",
    "model_name": "opsd_recurrent_example",
    "target_id": [
        "DE_load_actual_entsoe_transparency"
    ],
    "target_list": null,
    "start_date": null,
    "history_horizon": 147,
    "forecast_horizon": 24,
    "cap_limit": 1,
    "train_split": 0.6,
    "validation_split": 0.8,
    "periodicity": 24,
    "optimizer_name": "adam",
    "exploration": false,
    "cuda_id": null,
    "feature_groups": [
        {
            "name": "main",
            "scaler": [
                "minmax",
                0.0,
                1.0
            ],
            "features": [
                "DE_load_actual_entsoe_transparency",
                "DE_temperature",
                "DE_radiation_direct_horizontal",
                "DE_radiation_diffuse_horizontal"
            ]
        },
        {
            "name": "aux",
            "scaler": null,
            "features": [
                "hour_sin",
                "weekday_sin",
                "mnth_sin",
                "hour_cos",
                "weekday_cos",
                "mnth_cos"
            ]
        }
    ],
    "encoder_features": [
        "DE_load_actual_entsoe_transparency",
        "DE_temperature",
        "DE_radiation_direct_horizontal",
        "DE_radiation_diffuse_horizontal"
    ],
    "decoder_features": [
        "hour_sin",
        "weekday_sin",
        "mnth_sin",
        "hour_cos",
        "weekday_cos",
        "mnth_cos"
    ],
    "max_epochs": 50,
    "batch_size": 32,
    "learning_rate": 9.9027931032814e-05,
    "early_stopping_patience": 7,
    "early_stopping_margin": 0.0,
    "model_class": "recurrent",
    "model_parameters": {
        "recurrent": {
            "core_net": "torch.nn.LSTM",
            "core_layers": 1,
            "dropout_fc": 0.4,
            "dropout_core": 0.3,
            "rel_linear_hidden_size": 1.0,
            "rel_core_hidden_size": 1.0,
            "relu_leak": 0.1
        },
        "simple_transformer": {
            "num_layers": 3,
            "dropout": 0.4,
            "n_heads": 6
        }
    }
}

now train the model using the train.py skript. For more information on how to train a model, take a look at the other notebooks or at the user documentation (http://sogno.energy/proloaf/docs/overview/) 

In [ ]:
python ./src/train.py -s opsd_example

You should now have a trained model in the "oracles" folder

## configure the saliency map handler

 now the parameters for the saliency map calculation should be configured. The  saliency.json file should be in the targets folder (e.g targets/opsd_example).
 
 The following Parameters can be configured:
 
 - rel_interpretation_path: the path, where the output files are going to be saved to
 - date: The date for which the saliency map should be created. This is the date of the first timestep of the forecasting horizon
 - ref_batch_size: The number of different random noise time series to be added to the original input data. This determines the number of references.
 - max_epochs: The maximum number of epochs for the optimization of the objective function.
 - n_trials: The number of trials of the optuna study. This determines the number of trials with different hyperparameters. The only current hyperparameter is the learning rate of the optimizer.
 - lr_low: The lowest learning rate a study trial can take
 - lr_high The highest learning rate a study trial can take 
 - lambda: the weight that determines the size of the mask weight error. A high lambda value means that high mask values are penalized more in the objective function, so that the mask values turn out to be smaller overall.
 - cuda_id: the cuda id, which determines what cuda device is going to be used for the pytorch module (https://pytorch.org/)
 
 The config file should be in json format could looks like this:

In [ ]:
{
    "rel_interpretation_path": "oracles/interpretation/",
    "date": "27.07.2019 00:00:00",
    "ref_batch_size": 2,
    "max_epochs": 1,
    "n_trials": 1,
    "lr_low": 1e-6,
    "lr_high": 0.1,
    "lambda": 1,
    "cuda_id" : 0
}

# Run the Saliency Map optimizer

Import the Saliency Map Handler class

In [35]:
import src.proloaf.saliency_map as sm

First an instance of the saliency map handler has to be made. The name of the example targe5t station is opsd_example

In [37]:
target_station_name = 'opsd_example'
saliency_map_handler = sm.SaliencyMapHandler(target_station_name)

2022-05-17 19:07:09 - saliency_map - INFO - reading saliency.json config
2022-05-17 19:07:09 - saliency_map - INFO - reading model config...
2022-05-17 19:07:09 - saliency_map - INFO - importing data...
2022-05-17 19:07:09 - saliency_map - INFO - setting computation device...
2022-05-17 19:07:09 - saliency_map - DEBUG - Device: cpu
2022-05-17 19:07:09 - saliency_map - INFO - preparing the dataset...
2022-05-17 19:07:09 - tensorloader - DEBUG - frame not prepared
2022-05-17 19:07:09 - datahandler - INFO - No missing data
2022-05-17 19:07:10 - saliency_map - INFO - loading the forecasting model
2022-05-17 19:07:10 - saliency_map - ERROR - An error has occurred while trying to load the forecasting model.The model has to be trained and saved as a loadable file.
2022-05-17 19:07:10 - saliency_map - DEBUG - initializing saliency map...
2022-05-17 19:07:10 - saliency_map - INFO - Date and Time set to: 2019-07-27 00:00:00
2022-05-17 19:07:10 - saliency_map - DEBUG - Timestep for saliency map w

AttributeError: 'SaliencyMapHandler' object has no attribute '_model_wrap'

now that we have a class instance, the create_saliency_map() function can be called.

In [ ]:
saliency_map_handler.create_saliency_map()

at any point now the class instance can be saved, so that the results can be loaded again at any point in the future

In [ ]:
saliency_map_handler.save()

if you want to load a previously saved class instance call load()

In [ ]:
saliency_map_handler.load()

The visualize the result and save a plot, the plot() function should be called. 

In [ ]:
saliency_map_handler.plot()

If you also want to how the perturbed prediction performed in comparison to the unperturbed one use plot_predictions()

In [ ]:
saliency_map_handler.plot_predictions()